In [3]:
import pandas as pd
import numpy as np
from datetime import timedelta
from glob import glob

#pip install plotly
import plotly.express as px

pd.options.plotting.backend='plotly'

import plotly.io as pio
pio.renderers.default="notebook_connected"

import warnings
warnings.filterwarnings('ignore')

import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

In [4]:
#pip install cufflinks

In [5]:
#pip install chart_studio

In [6]:
site_info = pd.read_csv("site_info.csv") # 발전소 정보
energy = pd.read_csv("energy.csv") # 발전소별 발전량

dangjin_fcst_data = pd.read_csv("dangjin_fcst_data.csv") # 당진 예보 데이터
dangjin_obs_data = pd.read_csv("dangjin_obs_data.csv") # 당진 기상 관측 자료

ulsan_fcst_data = pd.read_csv("ulsan_fcst_data.csv") # 울산 예보 데이터
ulsan_obs_data = pd.read_csv("ulsan_obs_data.csv") # 울산 기상 관측 자료

sample_submission = pd.read_csv("sample_submission.csv") # 제출 양식

In [7]:
dangjin_fcst_data.describe()

,forecast,Temperature,Humidity,WindSpeed,WindDirection,Cloud
count,162208.000000,162208.000000,162208.000000,162208.000000,162208.000000,162208.000000
mean,30.675676,12.624634,76.996147,3.639262,214.147761,2.439411
std,16.714755,9.816256,17.435418,2.627592,108.020035,1.214753
min,4.000000,-14.000000,5.000000,0.000000,0.000000,1.000000
25%,16.000000,4.000000,65.000000,1.900000,115.000000,1.000000
50%,31.000000,13.000000,80.000000,2.800000,237.000000,3.000000
75%,43.000000,21.000000,90.000000,4.600000,313.000000,4.000000
max,67.000000,36.000000,100.000000,32.299999,360.000000,4.000000


In [8]:
energy_pure=energy.iloc[:,1:]
energy['total']=energy_pure.sum(axis=1)

In [9]:
energy.head(20)

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan,total
0,2018-03-01 1:00:00,0.0,0.0,0,0,0.0
1,2018-03-01 2:00:00,0.0,0.0,0,0,0.0
2,2018-03-01 3:00:00,0.0,0.0,0,0,0.0
3,2018-03-01 4:00:00,0.0,0.0,0,0,0.0
4,2018-03-01 5:00:00,0.0,0.0,0,0,0.0
5,2018-03-01 6:00:00,0.0,0.0,0,0,0.0
6,2018-03-01 7:00:00,0.0,0.0,0,0,0.0
7,2018-03-01 8:00:00,0.0,0.0,0,4,4.0
8,2018-03-01 9:00:00,36.0,33.0,37,35,141.0
9,2018-03-01 10:00:00,313.0,209.0,318,71,911.0


In [10]:
energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25632 entries, 0 to 25631
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   time               25632 non-null  object 
 1   dangjin_floating   25608 non-null  float64
 2   dangjin_warehouse  25584 non-null  float64
 3   dangjin            25632 non-null  int64  
 4   ulsan              25632 non-null  int64  
 5   total              25632 non-null  float64
dtypes: float64(3), int64(2), object(1)
memory usage: 1.2+ MB


### 당진 floating 과 당진 warehouse에는 na가 존재한다 
### na를 살펴보고 시작한다

In [11]:
# 당진floating의 na값을 살펴본다
# 2020-06-26 하루 동안 측정이 되지 않았음을 알 수 있다.
energy[energy['dangjin_floating'].isnull()]

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan,total
20352,2020-06-26 1:00:00,NaN,0.0,0,0,0.0
20353,2020-06-26 2:00:00,NaN,0.0,0,0,0.0
20354,2020-06-26 3:00:00,NaN,0.0,0,0,0.0
20355,2020-06-26 4:00:00,NaN,0.0,0,0,0.0
20356,2020-06-26 5:00:00,NaN,0.0,0,0,0.0
20357,2020-06-26 6:00:00,NaN,0.0,0,0,0.0
20358,2020-06-26 7:00:00,NaN,7.0,2,7,16.0
20359,2020-06-26 8:00:00,NaN,20.0,18,50,88.0
20360,2020-06-26 9:00:00,NaN,148.0,175,161,484.0
20361,2020-06-26 10:00:00,NaN,242.0,330,170,742.0


In [12]:
# 당진 warehouse의 na값은 2018-03-17 & 2018-03-18 에 측정이 안됐음을 알 수 있다.
energy[energy['dangjin_warehouse'].isnull()]

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan,total
384,2018-03-17 1:00:00,0.0,NaN,0,0,0.0
385,2018-03-17 2:00:00,0.0,NaN,0,0,0.0
386,2018-03-17 3:00:00,0.0,NaN,0,0,0.0
387,2018-03-17 4:00:00,0.0,NaN,0,0,0.0
388,2018-03-17 5:00:00,0.0,NaN,0,0,0.0
389,2018-03-17 6:00:00,0.0,NaN,0,0,0.0
390,2018-03-17 7:00:00,0.0,NaN,0,0,0.0
391,2018-03-17 8:00:00,24.0,NaN,16,13,53.0
392,2018-03-17 9:00:00,162.0,NaN,177,57,396.0
393,2018-03-17 10:00:00,363.0,NaN,348,206,917.0


### 원본데이터를 참고하려했으나 원본데이터에도 존재하지 않는다

## 우선 울산에 focus를 맞춘다

In [13]:
# 평가 기준이 되는 10% capacity 생성
total=1000+700+1000+500
LB=0.1*total
LB

320.0

In [14]:
# 울산 생산량 중에 1자리일 댸 total이 항상 10% 미만이지 않을까 싶어 eda해본다
# 하지만 당진과 울산의 기상상황이 달라서인지 total이상의 생산량이 많이 보인다.

In [15]:
ulsan_E=energy[energy['ulsan']<10]
ulsan_E=ulsan_E[ulsan_E['ulsan']>0]

In [16]:
ulsan_E['total'].iplot(kind='line')

In [17]:
# 시간으로 분류를해본다

energy['date'] = energy['time'].apply(lambda x: x.split()[0])
energy['time'] = energy['time'].apply(lambda x: x.split()[1])
energy['time'] = energy['time'].str.rjust(8,'0') # 한자릿수 시간 앞에 0 추가 ex) 3시 -> 03시

# 24시를 00시로 바꿔주기
energy.loc[energy['time']=='24:00:00','time'] = '00:00:00'
energy['time'] = energy['date'] + ' ' + energy['time']
energy['time'] = pd.to_datetime(energy['time'])
energy.loc[energy['time'].dt.hour==0,'time'] += timedelta(days=1)



energy['year']=energy['time'].dt.year
energy['month']=energy['time'].dt.month
energy['day']=energy['time'].dt.day
energy['hour']=energy['time'].dt.hour

In [18]:
energy.head(10)

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan,total,date,year,month,day,hour
0,2018-03-01 01:00:00,0.0,0.0,0,0,0.0,2018-03-01,2018,3,1,1
1,2018-03-01 02:00:00,0.0,0.0,0,0,0.0,2018-03-01,2018,3,1,2
2,2018-03-01 03:00:00,0.0,0.0,0,0,0.0,2018-03-01,2018,3,1,3
3,2018-03-01 04:00:00,0.0,0.0,0,0,0.0,2018-03-01,2018,3,1,4
4,2018-03-01 05:00:00,0.0,0.0,0,0,0.0,2018-03-01,2018,3,1,5
5,2018-03-01 06:00:00,0.0,0.0,0,0,0.0,2018-03-01,2018,3,1,6
6,2018-03-01 07:00:00,0.0,0.0,0,0,0.0,2018-03-01,2018,3,1,7
7,2018-03-01 08:00:00,0.0,0.0,0,4,4.0,2018-03-01,2018,3,1,8
8,2018-03-01 09:00:00,36.0,33.0,37,35,141.0,2018-03-01,2018,3,1,9
9,2018-03-01 10:00:00,313.0,209.0,318,71,911.0,2018-03-01,2018,3,1,10


###  시간대 별로 groupby 하여 시간대별 total값의 max를 추출한다.
### 만약 max값이 LB(평가기준)을 넘지 못하면 해당 시간대를 제외한다.

In [19]:
max_total_hour=energy['total'].groupby([energy['hour']]).max()
max_total_hour

hour
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6       10.0
7      217.0
8      447.0
9      884.0
10    1460.0
11    1918.0
12    2349.0
13    2513.0
14    2505.0
15    2383.0
16    2037.0
17    1506.0
18     908.0
19     369.0
20      57.0
21       0.0
22       0.0
23       0.0
Name: total, dtype: float64

## 20시부터 ~06시는 확실히 제거를 해도된다고 판단
## 07시는 한 번 더 데이터를 살펴보고 결정

In [20]:
energy_07=energy[energy['hour']==7]
energy_07[energy_07['month']==1].describe()

,dangjin_floating,dangjin_warehouse,dangjin,ulsan,total,year,month,day,hour
count,93.0,93.0,93.0,93.0,93.0,93.000000,93.0,93.000000,93.0
mean,0.0,0.0,0.0,0.0,0.0,2020.000000,1.0,16.000000,7.0
std,0.0,0.0,0.0,0.0,0.0,0.820922,0.0,8.992751,0.0
min,0.0,0.0,0.0,0.0,0.0,2019.000000,1.0,1.000000,7.0
25%,0.0,0.0,0.0,0.0,0.0,2019.000000,1.0,8.000000,7.0
50%,0.0,0.0,0.0,0.0,0.0,2020.000000,1.0,16.000000,7.0
75%,0.0,0.0,0.0,0.0,0.0,2021.000000,1.0,24.000000,7.0
max,0.0,0.0,0.0,0.0,0.0,2021.000000,1.0,31.000000,7.0


In [21]:
energy_07[energy_07['month']==1].describe()

,dangjin_floating,dangjin_warehouse,dangjin,ulsan,total,year,month,day,hour
count,93.0,93.0,93.0,93.0,93.0,93.000000,93.0,93.000000,93.0
mean,0.0,0.0,0.0,0.0,0.0,2020.000000,1.0,16.000000,7.0
std,0.0,0.0,0.0,0.0,0.0,0.820922,0.0,8.992751,0.0
min,0.0,0.0,0.0,0.0,0.0,2019.000000,1.0,1.000000,7.0
25%,0.0,0.0,0.0,0.0,0.0,2019.000000,1.0,8.000000,7.0
50%,0.0,0.0,0.0,0.0,0.0,2020.000000,1.0,16.000000,7.0
75%,0.0,0.0,0.0,0.0,0.0,2021.000000,1.0,24.000000,7.0
max,0.0,0.0,0.0,0.0,0.0,2021.000000,1.0,31.000000,7.0


In [22]:
energy_07[energy_07['month']==2].describe()

,dangjin_floating,dangjin_warehouse,dangjin,ulsan,total,year,month,day,hour
count,57.0,57.0,57.0,57.0,57.0,57.000000,57.0,57.000000,57.0
mean,0.0,0.0,0.0,0.0,0.0,2019.508772,2.0,14.754386,7.0
std,0.0,0.0,0.0,0.0,0.0,0.504367,0.0,8.302927,0.0
min,0.0,0.0,0.0,0.0,0.0,2019.000000,2.0,1.000000,7.0
25%,0.0,0.0,0.0,0.0,0.0,2019.000000,2.0,8.000000,7.0
50%,0.0,0.0,0.0,0.0,0.0,2020.000000,2.0,15.000000,7.0
75%,0.0,0.0,0.0,0.0,0.0,2020.000000,2.0,22.000000,7.0
max,0.0,0.0,0.0,0.0,0.0,2020.000000,2.0,29.000000,7.0


In [23]:
energy_07[energy_07['month']==3].describe()

,dangjin_floating,dangjin_warehouse,dangjin,ulsan,total,year,month,day,hour
count,93.000000,91.000000,93.0,93.000000,93.000000,93.000000,93.0,93.000000,93.0
mean,0.258065,0.087912,0.0,0.311828,0.655914,2019.000000,3.0,16.000000,7.0
std,0.674105,0.284736,0.0,0.675318,1.410324,0.820922,0.0,8.992751,0.0
min,0.000000,0.000000,0.0,0.000000,0.000000,2018.000000,3.0,1.000000,7.0
25%,0.000000,0.000000,0.0,0.000000,0.000000,2018.000000,3.0,8.000000,7.0
50%,0.000000,0.000000,0.0,0.000000,0.000000,2019.000000,3.0,16.000000,7.0
75%,0.000000,0.000000,0.0,0.000000,1.000000,2020.000000,3.0,24.000000,7.0
max,3.000000,1.000000,0.0,3.000000,7.000000,2020.000000,3.0,31.000000,7.0


In [24]:
energy_07[energy_07['month']==4].describe()

,dangjin_floating,dangjin_warehouse,dangjin,ulsan,total,year,month,day,hour
count,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.0,90.000000,90.0
mean,7.322222,6.144444,2.166667,6.366667,22.000000,2019.000000,4.0,15.500000,7.0
std,5.803578,5.155037,4.155002,5.322097,18.021211,0.821071,0.0,8.703932,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,2018.000000,4.0,1.000000,7.0
25%,2.000000,2.000000,0.000000,2.000000,8.000000,2018.000000,4.0,8.000000,7.0
50%,6.500000,5.000000,0.000000,5.000000,17.500000,2019.000000,4.0,15.500000,7.0
75%,11.750000,10.000000,2.000000,11.000000,34.500000,2020.000000,4.0,23.000000,7.0
max,21.000000,19.000000,19.000000,19.000000,73.000000,2020.000000,4.0,30.000000,7.0


In [25]:
energy_07[energy_07['month']==5].describe()

,dangjin_floating,dangjin_warehouse,dangjin,ulsan,total,year,month,day,hour
count,93.000000,93.000000,93.000000,93.000000,93.000000,93.000000,93.0,93.000000,93.0
mean,17.795699,15.537634,11.172043,16.139785,60.645161,2019.000000,5.0,16.000000,7.0
std,10.977803,9.777480,12.547842,10.091528,35.403465,0.820922,0.0,8.992751,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,2018.000000,5.0,1.000000,7.0
25%,10.000000,9.000000,0.000000,6.000000,33.000000,2018.000000,5.0,8.000000,7.0
50%,20.000000,18.000000,7.000000,18.000000,60.000000,2019.000000,5.0,16.000000,7.0
75%,26.000000,22.000000,19.000000,24.000000,87.000000,2020.000000,5.0,24.000000,7.0
max,43.000000,37.000000,52.000000,35.000000,153.000000,2020.000000,5.0,31.000000,7.0


In [26]:
energy_07[energy_07['month']==6].describe()

,dangjin_floating,dangjin_warehouse,dangjin,ulsan,total,year,month,day,hour
count,89.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.0,90.000000,90.0
mean,19.101124,16.922222,15.266667,19.055556,70.133333,2019.000000,6.0,15.500000,7.0
std,12.752045,11.236205,15.034642,11.895954,40.991421,0.821071,0.0,8.703932,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,2018.000000,6.0,1.000000,7.0
25%,9.000000,7.000000,1.000000,6.250000,38.250000,2018.000000,6.0,8.000000,7.0
50%,19.000000,17.000000,12.500000,23.000000,70.500000,2019.000000,6.0,15.500000,7.0
75%,28.000000,24.000000,24.750000,28.000000,99.750000,2020.000000,6.0,23.000000,7.0
max,54.000000,62.000000,76.000000,45.000000,217.000000,2020.000000,6.0,30.000000,7.0


In [27]:
energy_07[energy_07['month']==7].describe()

,dangjin_floating,dangjin_warehouse,dangjin,ulsan,total,year,month,day,hour
count,93.000000,93.000000,93.000000,93.000000,93.000000,93.000000,93.0,93.000000,93.0
mean,12.301075,10.731183,8.569892,11.204301,42.806452,2019.000000,7.0,16.000000,7.0
std,11.041212,9.437598,12.706039,7.965359,34.285013,0.820922,0.0,8.992751,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,2018.000000,7.0,1.000000,7.0
25%,4.000000,3.000000,0.000000,4.000000,18.000000,2018.000000,7.0,8.000000,7.0
50%,11.000000,9.000000,2.000000,12.000000,39.000000,2019.000000,7.0,16.000000,7.0
75%,16.000000,16.000000,14.000000,18.000000,58.000000,2020.000000,7.0,24.000000,7.0
max,60.000000,41.000000,54.000000,28.000000,159.000000,2020.000000,7.0,31.000000,7.0


In [28]:
energy_07[energy_07['month']==8].describe()

,dangjin_floating,dangjin_warehouse,dangjin,ulsan,total,year,month,day,hour
count,93.000000,93.000000,93.000000,93.000000,93.000000,93.000000,93.0,93.000000,93.0
mean,6.010753,4.924731,2.397849,6.021505,19.354839,2019.000000,8.0,16.000000,7.0
std,7.617906,4.222704,4.314132,5.559908,14.144169,0.820922,0.0,8.992751,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,2018.000000,8.0,1.000000,7.0
25%,2.000000,2.000000,0.000000,1.000000,9.000000,2018.000000,8.0,8.000000,7.0
50%,4.000000,4.000000,0.000000,5.000000,17.000000,2019.000000,8.0,16.000000,7.0
75%,8.000000,7.000000,2.000000,10.000000,25.000000,2020.000000,8.0,24.000000,7.0
max,60.000000,19.000000,20.000000,23.000000,66.000000,2020.000000,8.0,31.000000,7.0


In [29]:
energy_07[energy_07['month']==9].describe()

,dangjin_floating,dangjin_warehouse,dangjin,ulsan,total,year,month,day,hour
count,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.0,90.000000,90.0
mean,1.600000,0.988889,0.122222,1.366667,4.077778,2019.000000,9.0,15.500000,7.0
std,2.092254,1.267524,0.804870,2.008130,4.176520,0.821071,0.0,8.703932,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,2018.000000,9.0,1.000000,7.0
25%,0.000000,0.000000,0.000000,0.000000,0.000000,2018.000000,9.0,8.000000,7.0
50%,1.000000,1.000000,0.000000,0.000000,3.500000,2019.000000,9.0,15.500000,7.0
75%,2.000000,2.000000,0.000000,2.000000,6.000000,2020.000000,9.0,23.000000,7.0
max,10.000000,7.000000,7.000000,9.000000,22.000000,2020.000000,9.0,30.000000,7.0


In [30]:
energy_07[energy_07['month']==10].describe()

,dangjin_floating,dangjin_warehouse,dangjin,ulsan,total,year,month,day,hour
count,93.000000,93.000000,93.000000,93.000000,93.000000,93.000000,93.0,93.000000,93.0
mean,0.268817,0.150538,0.043011,0.150538,0.612903,2019.000000,10.0,16.000000,7.0
std,1.294972,1.350895,0.414781,0.388594,3.057536,0.820922,0.0,8.992751,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,2018.000000,10.0,1.000000,7.0
25%,0.000000,0.000000,0.000000,0.000000,0.000000,2018.000000,10.0,8.000000,7.0
50%,0.000000,0.000000,0.000000,0.000000,0.000000,2019.000000,10.0,16.000000,7.0
75%,0.000000,0.000000,0.000000,0.000000,0.000000,2020.000000,10.0,24.000000,7.0
max,12.000000,13.000000,4.000000,2.000000,29.000000,2020.000000,10.0,31.000000,7.0


In [31]:
energy_07[energy_07['month']==11].describe()

,dangjin_floating,dangjin_warehouse,dangjin,ulsan,total,year,month,day,hour
count,90.0,90.0,90.0,90.0,90.0,90.000000,90.0,90.000000,90.0
mean,0.0,0.0,0.0,0.0,0.0,2019.000000,11.0,15.500000,7.0
std,0.0,0.0,0.0,0.0,0.0,0.821071,0.0,8.703932,0.0
min,0.0,0.0,0.0,0.0,0.0,2018.000000,11.0,1.000000,7.0
25%,0.0,0.0,0.0,0.0,0.0,2018.000000,11.0,8.000000,7.0
50%,0.0,0.0,0.0,0.0,0.0,2019.000000,11.0,15.500000,7.0
75%,0.0,0.0,0.0,0.0,0.0,2020.000000,11.0,23.000000,7.0
max,0.0,0.0,0.0,0.0,0.0,2020.000000,11.0,30.000000,7.0


In [32]:
energy_07[energy_07['month']==12].describe()

,dangjin_floating,dangjin_warehouse,dangjin,ulsan,total,year,month,day,hour
count,93.0,93.0,93.0,93.0,93.0,93.000000,93.0,93.000000,93.0
mean,0.0,0.0,0.0,0.0,0.0,2019.000000,12.0,16.000000,7.0
std,0.0,0.0,0.0,0.0,0.0,0.820922,0.0,8.992751,0.0
min,0.0,0.0,0.0,0.0,0.0,2018.000000,12.0,1.000000,7.0
25%,0.0,0.0,0.0,0.0,0.0,2018.000000,12.0,8.000000,7.0
50%,0.0,0.0,0.0,0.0,0.0,2019.000000,12.0,16.000000,7.0
75%,0.0,0.0,0.0,0.0,0.0,2020.000000,12.0,24.000000,7.0
max,0.0,0.0,0.0,0.0,0.0,2020.000000,12.0,31.000000,7.0


In [33]:
ul_to7=energy_07[['ulsan','total']]
ul_to7_6=ul_to7[energy_07['month']==6]
ul_to7_6

,ulsan,total
2214,28,67.0
2238,28,64.0
2262,31,144.0
2286,41,143.0
2310,3,73.0
...,...,...
20358,7,16.0
20382,23,76.0
20406,23,37.0
20430,17,79.0


In [34]:
fig=px.line(ul_to7_6)
fig.show()

In [35]:
ul_to7_200=ul_to7[ul_to7['total']>=200]
ul_to7_200

,ulsan,total
2670,25,217.0


### 200을 넘긴것도 한 번 발생했던 일이기에 7시도 고려하지 않고 데이터 분석을 실시한다.

In [36]:
energy_new=energy[energy['hour']>=7]
energy_new=energy_new[energy_new['hour']<20]
energy_new

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan,total,date,year,month,day,hour
6,2018-03-01 07:00:00,0.0,0.0,0,0,0.0,2018-03-01,2018,3,1,7
7,2018-03-01 08:00:00,0.0,0.0,0,4,4.0,2018-03-01,2018,3,1,8
8,2018-03-01 09:00:00,36.0,33.0,37,35,141.0,2018-03-01,2018,3,1,9
9,2018-03-01 10:00:00,313.0,209.0,318,71,911.0,2018-03-01,2018,3,1,10
10,2018-03-01 11:00:00,532.0,296.0,490,82,1400.0,2018-03-01,2018,3,1,11
...,...,...,...,...,...,...,...,...,...,...,...
25622,2021-01-31 15:00:00,297.0,256.0,352,259,1164.0,2021-01-31,2021,1,31,15
25623,2021-01-31 16:00:00,230.0,166.0,261,196,853.0,2021-01-31,2021,1,31,16
25624,2021-01-31 17:00:00,76.0,65.0,81,83,305.0,2021-01-31,2021,1,31,17
25625,2021-01-31 18:00:00,10.0,6.0,0,8,24.0,2021-01-31,2021,1,31,18


In [37]:
#energy가 target이므로 y로 적는다
ulsan_y=energy_new.iloc[:,4:]
# 인덱스 재설정
ulsan_y=ulsan_y.reset_index(drop=True)

ulsan_y

,ulsan,total,date,year,month,day,hour
0,0,0.0,2018-03-01,2018,3,1,7
1,4,4.0,2018-03-01,2018,3,1,8
2,35,141.0,2018-03-01,2018,3,1,9
3,71,911.0,2018-03-01,2018,3,1,10
4,82,1400.0,2018-03-01,2018,3,1,11
...,...,...,...,...,...,...,...
13879,259,1164.0,2021-01-31,2021,1,31,15
13880,196,853.0,2021-01-31,2021,1,31,16
13881,83,305.0,2021-01-31,2021,1,31,17
13882,8,24.0,2021-01-31,2021,1,31,18


# 시간별로 따로 모델을 작성하자 
### 태양의 각도가 고정 & 데이터가 다 섞이면 같은 온도라도 다양한 시각에서 바라봐야하지만 시간이 같다면 기온의 의미가 강해진다.

# 일조량과 일사량을 예측해서 사용하자. 
### 태양광 발전과 가장 면밀한 관계를 가진 두변수를 예측해서 사용해보자

# 미세먼지 예보 데이터를 활용해보자


## 우선 시간별로 데이터 정렬

In [38]:
ulsan2019=ulsan_y[ulsan_y['year']==2019]
ulsan2019_7=ulsan2019[ulsan2019['hour']==7]
ulsan2019_7

,ulsan,total,date,year,month,day,hour
3978,0,0.0,2019-01-01,2019,1,1,7
3991,0,0.0,2019-01-02,2019,1,2,7
4004,0,0.0,2019-01-03,2019,1,3,7
4017,0,0.0,2019-01-04,2019,1,4,7
4030,0,0.0,2019-01-05,2019,1,5,7
...,...,...,...,...,...,...,...
8658,0,0.0,2019-12-27,2019,12,27,7
8671,0,0.0,2019-12-28,2019,12,28,7
8684,0,0.0,2019-12-29,2019,12,29,7
8697,0,0.0,2019-12-30,2019,12,30,7


In [39]:
ulsan_energy2019_t=pd.DataFrame(np.zeros((13,365)))

In [40]:
ulsan_energy2019_t

,0,1,2,3,4,5,6,7,8,9,...,355,356,357,358,359,360,361,362,363,364
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
for i in list(range(13)):
    ulsan_energy2019_t.iloc[i,:]=list(ulsan2019[ulsan2019['hour']==i+7].ulsan)
        

In [42]:
ulsan_energy2019_t

,0,1,2,3,4,5,6,7,8,9,...,355,356,357,358,359,360,361,362,363,364
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
2,45.0,46.0,46.0,45.0,11.0,18.0,38.0,27.0,37.0,7.0,...,4.0,46.0,45.0,41.0,6.0,49.0,46.0,7.0,16.0,49.0
3,152.0,155.0,157.0,152.0,62.0,68.0,123.0,131.0,144.0,37.0,...,27.0,111.0,116.0,137.0,23.0,160.0,152.0,30.0,119.0,162.0
4,237.0,242.0,247.0,225.0,185.0,173.0,204.0,211.0,237.0,90.0,...,47.0,157.0,140.0,213.0,0.0,245.0,235.0,62.0,211.0,253.0
5,285.0,288.0,293.0,266.0,264.0,269.0,237.0,258.0,291.0,119.0,...,68.0,236.0,190.0,250.0,0.0,294.0,282.0,45.0,237.0,301.0
6,288.0,304.0,307.0,200.0,279.0,290.0,271.0,279.0,306.0,289.0,...,35.0,249.0,243.0,258.0,0.0,313.0,293.0,40.0,201.0,316.0
7,258.0,290.0,295.0,78.0,262.0,275.0,258.0,269.0,226.0,285.0,...,25.0,222.0,252.0,250.0,0.0,292.0,274.0,59.0,133.0,301.0
8,249.0,242.0,253.0,86.0,222.0,233.0,222.0,227.0,132.0,247.0,...,35.0,191.0,206.0,202.0,6.0,249.0,230.0,33.0,105.0,257.0
9,144.0,148.0,158.0,34.0,143.0,84.0,145.0,112.0,68.0,159.0,...,50.0,107.0,64.0,92.0,4.0,146.0,135.0,14.0,67.0,161.0


In [43]:
ulsan_energy2019_t.columns=ulsan2019_7['date']
ulsan_energy2019=ulsan_energy2019_t.T
ulsan_energy2019.columns=list(range(7,20))
ulsan_energy2019

,7,8,9,10,11,12,13,14,15,16,17,18,19
date,,,,,,,,,,,,,
2019-01-01,0.0,1.0,45.0,152.0,237.0,285.0,288.0,258.0,249.0,144.0,33.0,0.0,0.0
2019-01-02,0.0,1.0,46.0,155.0,242.0,288.0,304.0,290.0,242.0,148.0,36.0,0.0,0.0
2019-01-03,0.0,1.0,46.0,157.0,247.0,293.0,307.0,295.0,253.0,158.0,39.0,0.0,0.0
2019-01-04,0.0,1.0,45.0,152.0,225.0,266.0,200.0,78.0,86.0,34.0,9.0,0.0,0.0
2019-01-05,0.0,0.0,11.0,62.0,185.0,264.0,279.0,262.0,222.0,143.0,41.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,0.0,1.0,49.0,160.0,245.0,294.0,313.0,292.0,249.0,146.0,34.0,0.0,0.0
2019-12-28,0.0,0.0,46.0,152.0,235.0,282.0,293.0,274.0,230.0,135.0,31.0,0.0,0.0
2019-12-29,0.0,0.0,7.0,30.0,62.0,45.0,40.0,59.0,33.0,14.0,2.0,0.0,0.0


In [44]:
ulsan7=ulsan_energy2019[7]
ulsan7.plot()


### 울산 energy 2018년 시간대별 나열

In [45]:
# 2018 만들기
ulsan2018=ulsan_y[ulsan_y['year']==2018]
ulsan2018_7=ulsan2018[ulsan2018['hour']==7]

ulsan2018['date'].unique().shape




(306,)

In [46]:
ulsan_energy2018_t=pd.DataFrame(np.zeros((13,306)))
for i in list(range(13)):
    ulsan_energy2018_t.iloc[i,:]=list(ulsan2018[ulsan2018['hour']==i+7].ulsan)


In [47]:
ulsan_energy2018_t.columns=ulsan2018_7['date']
ulsan_energy2018=ulsan_energy2018_t.T
ulsan_energy2018.columns=list(range(7,20))
ulsan_energy2018

,7,8,9,10,11,12,13,14,15,16,17,18,19
date,,,,,,,,,,,,,
2018-03-01,0.0,4.0,35.0,71.0,82.0,334.0,372.0,346.0,318.0,258.0,160.0,30.0,0.0
2018-03-02,0.0,18.0,102.0,218.0,264.0,278.0,362.0,352.0,319.0,263.0,163.0,30.0,0.0
2018-03-03,0.0,14.0,75.0,103.0,167.0,143.0,168.0,256.0,311.0,250.0,151.0,31.0,0.0
2018-03-04,0.0,20.0,91.0,166.0,257.0,264.0,200.0,105.0,86.0,43.0,20.0,1.0,0.0
2018-03-05,0.0,0.0,0.0,3.0,0.0,8.0,14.0,21.0,22.0,21.0,16.0,9.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-27,0.0,1.0,50.0,159.0,239.0,269.0,302.0,267.0,252.0,150.0,26.0,0.0,0.0
2018-12-28,0.0,1.0,47.0,168.0,259.0,306.0,323.0,305.0,259.0,156.0,35.0,0.0,0.0
2018-12-29,0.0,1.0,49.0,164.0,251.0,303.0,322.0,306.0,258.0,158.0,37.0,0.0,0.0


### 2020 만들기


In [48]:
# 2020 만들기
ulsan2020=ulsan_y[ulsan_y['year']==2020]
ulsan2020_7=ulsan2020[ulsan2020['hour']==7]

ulsan2020['date'].unique().shape

(366,)

In [49]:
ulsan_energy2020_t=pd.DataFrame(np.zeros((13,366)))
for i in list(range(13)):
    ulsan_energy2020_t.iloc[i,:]=list(ulsan2020[ulsan2020['hour']==i+7].ulsan)

In [50]:
ulsan_energy2020_t.columns=ulsan2020_7['date']
ulsan_energy2020=ulsan_energy2020_t.T
ulsan_energy2020.columns=list(range(7,20))
ulsan_energy2020

,7,8,9,10,11,12,13,14,15,16,17,18,19
date,,,,,,,,,,,,,
2020-01-01,0.0,0.0,25.0,142.0,218.0,261.0,302.0,291.0,241.0,150.0,38.0,0.0,0.0
2020-01-02,0.0,0.0,37.0,132.0,214.0,264.0,276.0,255.0,206.0,114.0,32.0,0.0,0.0
2020-01-03,0.0,0.0,42.0,138.0,222.0,271.0,291.0,281.0,241.0,149.0,38.0,0.0,0.0
2020-01-04,0.0,0.0,36.0,50.0,73.0,96.0,240.0,281.0,230.0,143.0,38.0,0.0,0.0
2020-01-05,0.0,0.0,39.0,140.0,225.0,271.0,277.0,271.0,223.0,113.0,33.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27,0.0,0.0,13.0,79.0,134.0,173.0,118.0,57.0,32.0,11.0,0.0,0.0,0.0
2020-12-28,0.0,0.0,40.0,136.0,218.0,257.0,274.0,262.0,218.0,129.0,30.0,0.0,0.0
2020-12-29,0.0,0.0,21.0,70.0,101.0,202.0,252.0,209.0,65.0,19.0,3.0,0.0,0.0


### 2021 만들기

In [51]:
# 2020 만들기
ulsan2021=ulsan_y[ulsan_y['year']==2021]
ulsan2021_7=ulsan2021[ulsan2021['hour']==7]

ulsan2021['date'].unique().shape

(31,)

In [52]:
ulsan_energy2021_t=pd.DataFrame(np.zeros((13,31)))
for i in list(range(13)):
    ulsan_energy2021_t.iloc[i,:]=list(ulsan2021[ulsan2021['hour']==i+7].ulsan)

In [53]:
ulsan_energy2021_t.columns=ulsan2021_7['date']
ulsan_energy2021=ulsan_energy2021_t.T
ulsan_energy2021.columns=list(range(7,20))
ulsan_energy2021

,7,8,9,10,11,12,13,14,15,16,17,18,19
date,,,,,,,,,,,,,
2021-01-01,0.0,0.0,45.0,151.0,237.0,275.0,260.0,195.0,232.0,145.0,23.0,0.0,0.0
2021-01-02,0.0,0.0,33.0,135.0,211.0,259.0,283.0,277.0,233.0,151.0,40.0,0.0,0.0
2021-01-03,0.0,0.0,12.0,134.0,234.0,276.0,298.0,281.0,186.0,52.0,10.0,0.0,0.0
2021-01-04,0.0,0.0,35.0,128.0,206.0,258.0,276.0,263.0,219.0,134.0,36.0,1.0,0.0
2021-01-05,0.0,0.0,20.0,96.0,149.0,213.0,209.0,90.0,72.0,57.0,18.0,1.0,0.0
2021-01-06,0.0,0.0,47.0,157.0,240.0,277.0,286.0,297.0,254.0,147.0,36.0,1.0,0.0
2021-01-07,0.0,0.0,26.0,144.0,243.0,300.0,322.0,317.0,255.0,156.0,51.0,0.0,0.0
2021-01-08,0.0,0.0,43.0,141.0,220.0,295.0,321.0,308.0,263.0,177.0,50.0,1.0,0.0
2021-01-09,0.0,0.0,44.0,151.0,242.0,292.0,314.0,304.0,261.0,177.0,52.0,1.0,0.0


In [55]:
ulsan_energy_time=pd.concat([ulsan_energy2018,ulsan_energy2019,ulsan_energy2020,ulsan_energy2021])
ulsan_energy_time

,7,8,9,10,11,12,13,14,15,16,17,18,19
date,,,,,,,,,,,,,
2018-03-01,0.0,4.0,35.0,71.0,82.0,334.0,372.0,346.0,318.0,258.0,160.0,30.0,0.0
2018-03-02,0.0,18.0,102.0,218.0,264.0,278.0,362.0,352.0,319.0,263.0,163.0,30.0,0.0
2018-03-03,0.0,14.0,75.0,103.0,167.0,143.0,168.0,256.0,311.0,250.0,151.0,31.0,0.0
2018-03-04,0.0,20.0,91.0,166.0,257.0,264.0,200.0,105.0,86.0,43.0,20.0,1.0,0.0
2018-03-05,0.0,0.0,0.0,3.0,0.0,8.0,14.0,21.0,22.0,21.0,16.0,9.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-27,0.0,1.0,46.0,148.0,240.0,290.0,306.0,303.0,269.0,199.0,72.0,6.0,0.0
2021-01-28,0.0,0.0,17.0,75.0,160.0,249.0,294.0,294.0,256.0,130.0,77.0,8.0,0.0
2021-01-29,0.0,1.0,56.0,160.0,250.0,310.0,331.0,327.0,289.0,218.0,91.0,6.0,0.0


In [57]:
ulsan_energy_time.to_csv("C:\\Users\\sungb\\파이썬\\태양광 데이콘\\python\\ulsan_energy_time.csv",index=True)

# 시간대별 변수 정렬

# 예측시 사용할 변수자체가 예측 데이터이다. 
# 이는 예보의 성향을 가지고 있을 것으로 예상이 되기에 train시에도 예보 데이터를 사용하기로하자

## 가장 최근의 예보데이터 사용해서 데이터 구성 & 선형보간


In [177]:
ulsan_fcst=pd.read_csv('ulsan_fcst_data.csv')

In [178]:
ulsan_fcst['Forecasted_time']=pd.to_datetime(ulsan_fcst['Forecast time'])

In [179]:
# 예보시전에 forecast를 더하여 예보 시각을 나타내는 변수 생성
def to_date(x):
    return pd.DateOffset(hours=x)

ulsan_fcst['Forecasted_time'] = ulsan_fcst['Forecasted_time'] + ulsan_fcst['forecast'].map(to_date)

In [180]:
ulsan_fcst

,Forecast time,forecast,Temperature,Humidity,WindSpeed,WindDirection,Cloud,Forecasted_time
0,2018-03-01 11:00:00,4.0,8.0,20.0,14.0,298.0,2.0,2018-03-01 15:00:00
1,2018-03-01 11:00:00,7.0,4.0,20.0,4.3,298.0,2.0,2018-03-01 18:00:00
2,2018-03-01 11:00:00,10.0,3.0,30.0,1.9,309.0,2.0,2018-03-01 21:00:00
3,2018-03-01 11:00:00,13.0,0.0,40.0,1.5,318.0,2.0,2018-03-02 00:00:00
4,2018-03-01 11:00:00,16.0,-1.0,45.0,1.8,308.0,2.0,2018-03-02 03:00:00
...,...,...,...,...,...,...,...,...
162203,2021-03-01 08:00:00,52.0,9.0,45.0,3.1,83.0,3.0,2021-03-03 12:00:00
162204,2021-03-01 08:00:00,55.0,9.0,45.0,3.0,111.0,3.0,2021-03-03 15:00:00
162205,2021-03-01 08:00:00,58.0,8.0,55.0,2.2,122.0,3.0,2021-03-03 18:00:00
162206,2021-03-01 08:00:00,61.0,6.0,65.0,0.9,131.0,3.0,2021-03-03 21:00:00


In [184]:
## 가장 최근에 예보된 값으로 데이터 생성
sorted_fcst=ulsan_fcst.sort_values("Forecasted_time")
sorted_fcst.head(30)

,Forecast time,forecast,Temperature,Humidity,WindSpeed,WindDirection,Cloud,Forecasted_time
0,2018-03-01 11:00:00,4.0,8.0,20.0,14.0,298.0,2.0,2018-03-01 15:00:00
20,2018-03-01 14:00:00,4.0,4.0,20.0,4.3,298.0,2.0,2018-03-01 18:00:00
1,2018-03-01 11:00:00,7.0,4.0,20.0,4.3,298.0,2.0,2018-03-01 18:00:00
2,2018-03-01 11:00:00,10.0,3.0,30.0,1.9,309.0,2.0,2018-03-01 21:00:00
21,2018-03-01 14:00:00,7.0,3.0,30.0,1.9,309.0,2.0,2018-03-01 21:00:00
39,2018-03-01 17:00:00,4.0,3.0,30.0,2.1,313.0,2.0,2018-03-01 21:00:00
3,2018-03-01 11:00:00,13.0,0.0,40.0,1.5,318.0,2.0,2018-03-02 00:00:00
57,2018-03-01 20:00:00,4.0,0.0,40.0,1.3,321.0,2.0,2018-03-02 00:00:00
40,2018-03-01 17:00:00,7.0,0.0,40.0,1.3,321.0,2.0,2018-03-02 00:00:00
22,2018-03-01 14:00:00,10.0,0.0,40.0,1.5,318.0,2.0,2018-03-02 00:00:00


In [119]:
### 시간별 예보 최근 데이터를 담기위한 행렬을 만들자

last_time=pd.DataFrame(sorted_fcst.groupby(['Forecast_time'])['Forecast time'].max())
last_time=last_time.reset_index()
last_time

,Forecast_time,Forecast time
0,2018-03-01 15:00:00,2018-03-01 11:00:00
1,2018-03-01 18:00:00,2018-03-01 14:00:00
2,2018-03-01 21:00:00,2018-03-01 17:00:00
3,2018-03-02 00:00:00,2018-03-01 20:00:00
4,2018-03-02 03:00:00,2018-03-01 23:00:00
...,...,...
8783,2021-03-03 12:00:00,2021-03-01 08:00:00
8784,2021-03-03 15:00:00,2021-03-01 08:00:00
8785,2021-03-03 18:00:00,2021-03-01 08:00:00
8786,2021-03-03 21:00:00,2021-03-01 08:00:00


In [126]:
last_fcst=np.zeros((8788,5))
for i in list(range(last_fcst.shape[0])):
    last_fcst[i,:]=sorted_fcst[['Temperature','Humidity','WindSpeed','WindDirection','Cloud']][(sorted_fcst['Forecast time']==last_time.iloc[i,1])&(sorted_fcst['Forecast_time']==last_time.iloc[i,0])]

In [128]:
last_fcst=pd.DataFrame(last_fcst)

In [129]:
last_fcst['Forecast_time']=last_time['Forecast_time']
last_fcst['Forecast time']=last_time['Forecast time']

In [134]:
last_fcst.columns=['Temperature','Humidity','WindSpeed','WindDirection','Cloud','Forecast_time','Forecast time']

In [135]:
last_fcst

,Temperature,Humidity,WindSpeed,WindDirection,Cloud,Forecast_time,Forecast time
0,8.0,20.0,14.0,298.0,2.0,2018-03-01 15:00:00,2018-03-01 11:00:00
1,4.0,20.0,4.3,298.0,2.0,2018-03-01 18:00:00,2018-03-01 14:00:00
2,3.0,30.0,2.1,313.0,2.0,2018-03-01 21:00:00,2018-03-01 17:00:00
3,0.0,40.0,1.3,321.0,2.0,2018-03-02 00:00:00,2018-03-01 20:00:00
4,-1.0,45.0,1.8,299.0,2.0,2018-03-02 03:00:00,2018-03-01 23:00:00
...,...,...,...,...,...,...,...
8783,9.0,45.0,3.1,83.0,3.0,2021-03-03 12:00:00,2021-03-01 08:00:00
8784,9.0,45.0,3.0,111.0,3.0,2021-03-03 15:00:00,2021-03-01 08:00:00
8785,8.0,55.0,2.2,122.0,3.0,2021-03-03 18:00:00,2021-03-01 08:00:00
8786,6.0,65.0,0.9,131.0,3.0,2021-03-03 21:00:00,2021-03-01 08:00:00


In [132]:
last_fcst.isna().sum()

0                0
1                0
2                0
3                0
4                0
Forecast_time    0
Forecast time    0
dtype: int64

In [137]:
# 연산이 오래 걸리므로 데이터를 추출해둔다
last_fcst.to_csv("C:\\Users\\sungb\\파이썬\\태양광 데이콘\\python\\ulsan_last_fcst.csv",index=False)

In [138]:
last_fcst.describe()

,Temperature,Humidity,WindSpeed,WindDirection,Cloud
count,8788.000000,8788.000000,8788.000000,8788.000000,8788.000000
mean,15.057795,67.365157,3.536299,198.292899,2.441966
std,8.380721,19.499003,2.703466,113.774384,1.233121
min,-12.000000,5.000000,0.000000,0.000000,1.000000
25%,9.000000,55.000000,1.900000,68.000000,1.000000
50%,16.000000,70.000000,2.900000,229.000000,3.000000
75%,22.000000,85.000000,4.200000,300.000000,4.000000
max,35.000000,100.000000,36.099998,360.000000,4.000000


In [140]:
# 회귀로 갈거면 섭씨->화씨 전환
